In [64]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import yfinance as yf

# Mapping of ticker symbols to entity IDs
ticker_to_entity_id = {
    'MSTR': 1,
    'MARA': 4
}

# Function to fetch historical Bitcoin prices
def get_historical_bitcoin_prices(start_date, end_date):
    btc = yf.Ticker("BTC-USD")
    btc_hist = btc.history(start=start_date, end=end_date)
    btc_hist = btc_hist[['Open']].rename(columns={'Open': 'BTC Price'})
    btc_hist.reset_index(inplace=True)
    btc_hist['Date'] = pd.to_datetime(btc_hist['Date']).dt.tz_localize(None)
    return btc_hist

# Function to fetch historical Bitcoin holdings and interpolate data to daily frequency
def get_historical_bitcoin_holdings_daily(ticker):
    entity_id = ticker_to_entity_id.get(ticker)
    if not entity_id:
        raise ValueError(f"Ticker {ticker} not found in the mapping.")
    
    url = f"https://bitcointreasuries.net/entities/{entity_id}"
    response = requests.get(url)
    html_content = response.content

    # Parse the HTML content
    soup = BeautifulSoup(html_content, 'html.parser')

    # Find the table with the Bitcoin holdings
    table = soup.find("table", {"class": "relative min-w-full divide-y divide-gray-300 font-mono text-xs xs:text-base"})

    # Extract the rows from the table
    rows = table.find_all("tr", class_="cursor-pointer")

    # Extract the date and Bitcoin balance from each row
    holdings = []
    for row in rows:
        cols = row.find_all("td")
        if len(cols) > 2:
            date = cols[1].find('time').get('datetime')
            btc_balance = cols[2].get_text(strip=True)
            btc_balance = int(btc_balance.replace(',', ''))
            holdings.append((date, btc_balance))

    # Convert to DataFrame
    df = pd.DataFrame(holdings, columns=['Date', 'BTC Balance'])
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    # Create a complete date range from the earliest date to the most recent date
    complete_date_range = pd.date_range(start=df.index.min(), end=pd.Timestamp.today())

    # Reindex the DataFrame to include the complete date range
    df = df.reindex(complete_date_range)

    # Forward fill the missing values
    df['BTC Balance'] = df['BTC Balance'].ffill()

    # Slice the data to include only the range from 2022-01-01 to now
    df = df['2022-01-01':]

    # Reset the index to have 'Date' as a column again
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'Date'}, inplace=True)

    return df

# Function to calculate market value
def calculate_market_value(holdings_df, prices_df):
    # Merge holdings with prices
    merged_df = pd.merge(holdings_df, prices_df, on='Date', how='left')
    # Calculate market value
    merged_df['Market Value'] = merged_df['BTC Balance'] * merged_df['BTC Price']
    return merged_df

# Function to fetch historical stock data and shares outstanding
def get_historical_stock_data(ticker, start_date, end_date):
    stock = yf.Ticker(ticker)
    stock_hist = stock.history(start=start_date, end=end_date)
    shares_outstanding = stock.get_shares_full(start=start_date, end=end_date)
    
    stock_hist = stock_hist[['Close']].rename(columns={'Close': 'Stock Price'})
    stock_hist.reset_index(inplace=True)
    stock_hist['Date'] = pd.to_datetime(stock_hist['Date']).dt.tz_localize(None)
    
    shares_outstanding = shares_outstanding.reset_index()
    shares_outstanding.columns = ['Date', 'Shares Outstanding']
    shares_outstanding['Date'] = pd.to_datetime(shares_outstanding['Date']).dt.tz_localize(None)
    
    # Merge shares outstanding with stock price data
    stock_hist = pd.merge(stock_hist, shares_outstanding, on='Date', how='left')
    
    # Forward fill missing values for shares outstanding
    stock_hist['Shares Outstanding'] = stock_hist['Shares Outstanding'].ffill()
    
    # Calculate market cap
    stock_hist['Market Cap'] = stock_hist['Stock Price'] * stock_hist['Shares Outstanding']
    
    return stock_hist[['Date', 'Market Cap']]

# Function to merge market cap data
def merge_market_cap(holdings_df, stock_df):
    # Merge holdings with stock market cap
    merged_df = pd.merge(holdings_df, stock_df, on='Date', how='left')
    
    # Remove duplicate dates, keep the first occurrence
    merged_df = merged_df[~merged_df['Date'].duplicated(keep='first')]
    
    # Forward fill missing market cap values for weekends
    merged_df['Market Cap'] = merged_df['Market Cap'].ffill()
    return merged_df



In [63]:
# Main execution
# Fetch the historical Bitcoin holdings with daily data points for MicroStrategy and Marathon Digital
microstrategy_holdings = get_historical_bitcoin_holdings_daily('MSTR')
marathon_holdings = get_historical_bitcoin_holdings_daily('MARA')

# Fetch the historical Bitcoin prices
start_date = min(microstrategy_holdings['Date'].min(), marathon_holdings['Date'].min())
end_date = pd.Timestamp.today()
bitcoin_prices = get_historical_bitcoin_prices(start_date, end_date)

# Fetch the historical stock data and shares outstanding
microstrategy_stock_data = get_historical_stock_data('MSTR', start_date, end_date)
marathon_stock_data = get_historical_stock_data('MARA', start_date, end_date)

# Calculate the market value for each date for MicroStrategy
microstrategy_market_values = calculate_market_value(microstrategy_holdings, bitcoin_prices)

# Calculate the market value for each date for Marathon Digital
marathon_market_values = calculate_market_value(marathon_holdings, bitcoin_prices)

# Merge market cap data
microstrategy_final = merge_market_cap(microstrategy_market_values, microstrategy_stock_data)
marathon_final = merge_market_cap(marathon_market_values, marathon_stock_data)


# Print the results for Marathon Digital
print("Marathon Digital Historical Bitcoin Holdings, Market Values, and Market Cap:")
marathon_final

Marathon Digital Historical Bitcoin Holdings, Market Values, and Market Cap:


,Date,BTC Balance,BTC Price,Market Value,Market Cap
0,2022-01-01,8133.0,46311.746094,3.766534e+08,NaN
1,2022-01-02,8133.0,47680.925781,3.877890e+08,NaN
2,2022-01-03,8133.0,47343.542969,3.850450e+08,NaN
3,2022-01-04,8133.0,46458.851562,3.778498e+08,NaN
4,2022-01-05,8133.0,45899.359375,3.732995e+08,NaN
...,...,...,...,...,...
868,2024-05-10,17631.0,63055.191406,1.111726e+09,4.785821e+09
869,2024-05-11,17631.0,60793.355469,1.071848e+09,NaN
870,2024-05-12,17631.0,60793.503906,1.071850e+09,NaN
871,2024-05-13,17631.0,61451.218750,1.083446e+09,4.813711e+09
